# Feature engineering

## Library importation

In [24]:
# Traitement de données
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from scipy.stats import spearmanr
from sklearn.model_selection import train_test_split
from typing import List

## Download the dataset

In [25]:
X = pd.read_csv("X_train_NHkHMNU.csv")
y = pd.read_csv("y_train_ZAN5mwg.csv")

df = pd.concat([X, y], axis=1)

df = df.drop(df.columns[-2], axis=1)

## Remove columns that have -1 correlation

Some vairables have -1 correlation :
- `DE_NET_EXPORT` and `DE_NET_IMPORT`
- `FR_NET_EXPORT` and `FR_NET_IMPORT`
- `DE_FR_EXCHANGE` and `FR_DE_EXCHANGE`

Moreover they have the same correlation with the other variables. So keeping both variables doesn't add meaning full information. That is why we chose to drop one of the variables from each -1 correlation.

In [26]:
columns_name = ["DE_NET_IMPORT", "FR_NET_IMPORT", "DE_FR_EXCHANGE"]
for c in columns_name:
    df.drop(columns=c, inplace=True)

## Split the dataset

In [27]:
df_fr = df[df["COUNTRY"] == "FR"].copy()
df_de = df[df["COUNTRY"] == "DE"].copy()

## Change Nan Values from both dataset

In [28]:
# French dataset
numeric_cols_fr = df_fr.select_dtypes(include=["number"]).columns
df_fr[numeric_cols_fr] = df_fr[numeric_cols_fr].fillna(df_fr[numeric_cols_fr].median())

# German dataset
numeric_cols_de = df_de.select_dtypes(include=["number"]).columns
df_de[numeric_cols_de] = df_de[numeric_cols_de].fillna(df_de[numeric_cols_de].median())

## Create additionnal columns that represents a Threshold

##### Seuils pour df_fr
- COAL_RET < 0.8
- FR_CONSUMPTION > 1.5
- FR_NUCLEAR < -1.8
- FR_HYDRO < -0.4

##### Seuils pour df_de
- DE_CONSUMPTION > 1.2
- DE_NET_EXPORT > -0.45
- DE_WINDPOW > 0.3


Transformation "ReLU"

In [29]:
def AddSeuilColumn(df: pd.DataFrame, column_name: str, seuil: float, way: str):
    message = column_name + "_THRESHOLD_" + str(seuil)
    if way == "sup":
        df[message] = df[column_name].where(df[column_name] >= seuil, 0)
    else:
        df[message] = df[column_name].where(df[column_name] <= seuil, 0)

threshold_fr = {"COAL_RET": [0.8, "inf"],
                "FR_CONSUMPTION": [1.5, "sup"],
                "FR_NUCLEAR": [-1.8, "inf"],
                "FR_HYDRO":[-0.4, "inf"]                
                }

threshold_de = {"DE_CONSUMPTION": [1.2, "sup"],
                "DE_NET_EXPORT": [-0.45, "sup"],
                "DE_WINDPOW": [0.3, "sup"]
}

# add threshold columns to the french dataset
for key, value in threshold_fr.items():
    AddSeuilColumn(df_fr, key, value[0], value[1])

# add threshold columns to the german dataset
for key, value in threshold_de.items():
    AddSeuilColumn(df_de, key, value[0], value[1])

In [30]:
df_fr

,DAY_ID,COUNTRY,DE_CONSUMPTION,FR_CONSUMPTION,FR_DE_EXCHANGE,DE_NET_EXPORT,FR_NET_EXPORT,DE_GAS,FR_GAS,DE_COAL,...,DE_TEMP,FR_TEMP,GAS_RET,COAL_RET,CARBON_RET,TARGET,COAL_RET_THRESHOLD_0.8,FR_CONSUMPTION_THRESHOLD_1.5,FR_NUCLEAR_THRESHOLD_-1.8,FR_HYDRO_THRESHOLD_-0.4
0,206,FR,0.210099,-0.427458,0.606523,-0.244606,0.692860,0.441238,-0.213766,0.740627,...,-1.069070,-0.063404,0.339041,0.124552,-0.002445,0.028313,0.124552,0.000000,0.000000,0.000000
1,501,FR,-0.022399,-1.003452,0.022063,-0.573520,-1.130838,0.174773,0.426940,-0.170392,...,0.437419,1.831241,-0.659091,0.047114,-0.490365,-0.112516,0.047114,0.000000,-2.185961,-0.807112
2,687,FR,1.395035,1.978665,-1.021305,-0.622021,-1.682587,2.351913,2.122241,1.572267,...,0.684884,0.114836,0.535974,0.743338,0.204952,-0.180840,0.743338,1.978665,0.000000,0.000000
4,818,FR,0.143807,-0.617038,0.924990,-0.244606,0.990324,0.238693,-0.240862,1.003734,...,0.614338,0.729495,0.245109,1.526606,2.614378,-0.071733,0.000000,0.000000,0.000000,-0.795983
5,467,FR,-0.295296,-0.765120,0.717490,-1.117139,-0.200305,1.533595,0.306422,-1.240316,...,0.102046,0.472708,0.891049,0.861408,1.124457,0.932105,0.000000,0.000000,-1.920695,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1483,510,FR,0.422357,-0.704613,1.019784,-0.977214,1.112333,1.147306,-0.456156,-0.306209,...,-0.322520,-0.555211,1.946355,0.867074,1.322433,0.108277,0.000000,0.000000,0.000000,0.000000
1486,985,FR,0.117491,0.944372,-1.171116,1.403843,0.499653,-0.110824,1.320758,-1.024039,...,0.324165,0.829517,0.494188,1.011794,1.472650,0.827636,0.000000,0.000000,0.000000,0.000000
1487,905,FR,0.968724,0.459382,-0.996808,0.101161,-1.048997,2.360936,0.715357,0.189984,...,-2.413150,-2.069991,1.145686,0.335645,0.606318,0.049618,0.335645,0.000000,0.000000,0.000000
1490,887,FR,1.618582,1.752840,-0.611392,0.449153,-0.152146,1.972779,1.558300,0.561356,...,-0.009017,-0.012600,0.932633,-0.085690,0.356356,-0.063546,-0.085690,1.752840,0.000000,0.000000


In [31]:
df_de

,DAY_ID,COUNTRY,DE_CONSUMPTION,FR_CONSUMPTION,FR_DE_EXCHANGE,DE_NET_EXPORT,FR_NET_EXPORT,DE_GAS,FR_GAS,DE_COAL,...,FR_WIND,DE_TEMP,FR_TEMP,GAS_RET,COAL_RET,CARBON_RET,TARGET,DE_CONSUMPTION_THRESHOLD_1.2,DE_NET_EXPORT_THRESHOLD_-0.45,DE_WINDPOW_THRESHOLD_0.3
3,720,DE,-0.983324,-0.849198,0.839586,-0.270870,0.563230,0.487818,0.194659,-1.473817,...,-0.236249,0.350938,-0.417514,0.911652,-0.296168,1.073948,-0.260356,0.000000,-0.270870,0.000000
11,116,DE,-0.055692,-0.811337,-0.237105,-0.851082,-1.091142,0.882313,-0.145637,-0.042992,...,1.777992,1.595158,0.158463,-0.359866,-0.203952,-0.376234,-0.133381,0.000000,0.000000,0.000000
12,406,DE,0.532116,-0.331101,-0.339942,-0.173123,-1.312029,-0.188430,1.382599,-0.354327,...,0.100498,1.241892,-0.206340,1.170760,0.133643,0.033874,0.196312,0.000000,-0.173123,1.448078
13,1175,DE,-0.328286,-1.062255,1.380464,-1.046122,1.002243,0.544008,-0.730992,-1.183566,...,-0.695013,-0.634046,-0.168491,0.122818,0.220077,5.453331,-0.025477,0.000000,0.000000,0.359210
14,309,DE,1.028987,1.629315,-1.129663,-0.391261,-1.823117,2.170761,2.122272,1.831623,...,-0.945562,-0.667496,-1.566773,0.689483,1.095473,0.342798,0.460278,0.000000,-0.391261,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1485,78,DE,0.810509,-0.235078,-0.962203,0.971934,-0.121857,0.187811,0.219593,-0.605620,...,-0.456403,1.400533,1.512197,0.793561,0.848558,0.517578,-0.015261,0.000000,0.971934,1.619323
1488,87,DE,-0.295522,-1.051247,1.651718,-1.969871,0.074044,1.172127,0.076716,-0.700021,...,-0.564584,1.120555,0.865314,-0.423802,0.067325,1.558886,0.110206,0.000000,0.000000,0.000000
1489,809,DE,1.529204,1.106682,1.855327,-0.218658,1.450426,1.810665,1.388269,0.359723,...,-0.050781,-0.035360,-0.032517,0.876984,0.819520,1.320373,-0.172597,1.529204,-0.218658,0.000000
1491,1083,DE,0.856399,0.489199,0.255778,-1.531544,-0.829568,2.108764,1.866399,1.072553,...,0.894011,0.256338,0.402316,-1.112899,-0.237835,0.067152,0.151797,0.000000,0.000000,0.000000


## Remove Columns that have a low correlation with the TARGET variable

In [32]:
# COLONNES RECUPEREES TEMPORAIREMENT A LA MAIN CAR SEPARATIONN DES FICHIERS ANALYSES ET ENGINEERING
# A RECUPER DES VARIBALES QUAND LE RASSEMBLEMENT DES FICHIERS SERA FAIT

columns_keep_fr = ["DE_CONSUMPTION",
                "DE_NET_EXPORT",
                "DE_NET_IMPORT",
                "FR_COAL",
                "DE_HYDRO",
                "FR_HYDRO",
                "DE_WINDPOW",
                "FR_WINDPOW",
                "DE_RESIDUAL_LOAD",
                "DE_RAIN",
                "GAS_RET",
                "COAL_RET",
                "CARBON_RET"]

columns_keep_de = ["DE_CONSUMPTION",
                   "DE_FR_EXCHANGE",
                   "FR_DE_EXCHANGE",
                   "DE_NET_EXPORT",
                   "DE_NET_IMPORT",
                   "DE_GAS",
                   "FR_GAS",
                   "DE_COAL",
                   "DE_HYDRO",
                   "FR_HYDRO",
                   "DE_WINDPOW",
                   "FR_WINDPOW",
                   "DE_LIGNITE",
                   "DE_RESIDUAL_LOAD",
                   "FR_RAIN",
                   "DE_WIND",
                   "FR_WIND"]

# drop columns that are not in thoses lists
# french
for c in df_fr.columns:
    if c not in columns_keep_fr and "_THRESHOLD_" not in c:
        df_fr.drop(columns=c, inplace=True)

#german
for c in df_de.columns:
    if c not in columns_keep_de and "_THRESHOLD_" not in c:
        df_de.drop(columns=c, inplace=True)

In [33]:
df_fr

,DE_CONSUMPTION,DE_NET_EXPORT,FR_COAL,DE_HYDRO,FR_HYDRO,DE_WINDPOW,FR_WINDPOW,DE_RESIDUAL_LOAD,DE_RAIN,GAS_RET,COAL_RET,CARBON_RET,COAL_RET_THRESHOLD_0.8,FR_CONSUMPTION_THRESHOLD_1.5,FR_NUCLEAR_THRESHOLD_-1.8,FR_HYDRO_THRESHOLD_-0.4
0,0.210099,-0.244606,0.288782,2.209047,0.207838,-0.573370,-0.269460,0.626666,-0.172680,0.339041,0.124552,-0.002445,0.124552,0.000000,0.000000,0.000000
1,-0.022399,-0.573520,-0.762153,0.187964,-0.807112,-0.035514,-0.107350,-0.395469,-1.240300,-0.659091,0.047114,-0.490365,0.047114,0.000000,-2.185961,-0.807112
2,1.395035,-0.622021,0.777053,-0.108578,0.779142,-0.298755,-0.141239,1.336625,-0.480700,0.535974,0.743338,0.204952,0.743338,1.978665,0.000000,0.000000
4,0.143807,-0.244606,-0.274975,-0.230179,-0.795983,-0.774941,-0.564498,0.571613,-0.541465,0.245109,1.526606,2.614378,0.000000,0.000000,0.000000,-0.795983
5,-0.295296,-1.117139,-0.775944,2.306980,0.593251,-0.977976,-0.245628,-0.001279,-0.962519,0.891049,0.861408,1.124457,0.000000,0.000000,-1.920695,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1483,0.422357,-0.977214,-0.793961,1.781299,0.538795,-0.578489,0.149270,0.526433,1.848129,1.946355,0.867074,1.322433,0.000000,0.000000,0.000000,0.000000
1486,0.117491,1.403843,-0.779539,-0.385397,0.375729,2.285474,1.646472,-1.504396,-0.371909,0.494188,1.011794,1.472650,0.000000,0.000000,0.000000,0.000000
1487,0.968724,0.101161,-0.305680,-0.315249,-0.188248,-0.446788,0.536297,0.668574,-0.291708,1.145686,0.335645,0.606318,0.335645,0.000000,0.000000,0.000000
1490,1.618582,0.449153,0.230746,-0.341147,2.957114,0.198857,0.789618,1.358927,-0.268040,0.932633,-0.085690,0.356356,-0.085690,1.752840,0.000000,0.000000


In [34]:
df_de

,DE_CONSUMPTION,FR_DE_EXCHANGE,DE_NET_EXPORT,DE_GAS,FR_GAS,DE_COAL,DE_HYDRO,FR_HYDRO,DE_WINDPOW,FR_WINDPOW,DE_LIGNITE,DE_RESIDUAL_LOAD,FR_RAIN,DE_WIND,FR_WIND,DE_CONSUMPTION_THRESHOLD_1.2,DE_NET_EXPORT_THRESHOLD_-0.45,DE_WINDPOW_THRESHOLD_0.3
3,-0.983324,0.839586,-0.270870,0.487818,0.194659,-1.473817,-0.368417,1.320483,-0.010090,0.366885,-2.330557,-1.191889,-0.507570,-0.499409,-0.236249,0.000000,-0.270870,0.000000
11,-0.055692,-0.237105,-0.851082,0.882313,-0.145637,-0.042992,1.282374,-0.742765,-0.103994,0.356181,-0.354480,-0.178397,2.014508,1.143607,1.777992,0.000000,0.000000,0.000000
12,0.532116,-0.339942,-0.173123,-0.188430,1.382599,-0.354327,-0.168264,-1.071334,1.448078,0.079753,-0.237658,-0.795593,-1.394500,1.383171,0.100498,0.000000,-0.173123,1.448078
13,-0.328286,1.380464,-1.046122,0.544008,-0.730992,-1.183566,0.523610,1.061971,0.359210,-0.304661,-2.244028,-0.676137,-0.069416,-0.138918,-0.695013,0.000000,0.000000,0.359210
14,1.028987,-1.129663,-0.391261,2.170761,2.122272,1.831623,1.223032,1.885952,-1.210165,0.326872,0.892261,2.156285,-0.516738,-1.106067,-0.945562,0.000000,-0.391261,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1485,0.810509,-0.962203,0.971934,0.187811,0.219593,-0.605620,0.128621,0.154342,1.619323,0.386094,0.103651,-0.696438,-0.008048,-0.459455,-0.456403,0.000000,0.971934,1.619323
1488,-0.295522,1.651718,-1.969871,1.172127,0.076716,-0.700021,2.053830,-0.359158,-1.010181,-0.657622,-0.445943,0.604003,0.854559,-0.721643,-0.564584,0.000000,0.000000,0.000000
1489,1.529204,1.855327,-0.218658,1.810665,1.388269,0.359723,-0.470809,1.708814,-0.057214,2.968535,0.790548,1.547782,-0.230959,-0.098259,-0.050781,1.529204,-0.218658,0.000000
1491,0.856399,0.255778,-1.531544,2.108764,1.866399,1.072553,0.037892,0.277630,-0.981718,0.303324,0.211422,1.493870,0.404763,-0.594595,0.894011,0.000000,0.000000,0.000000


**QUESTIONS**

FAIRE UN PCQ POUR LES VARIABLES NON SEUIL ??

## Modèle de base

In [35]:
X_all = df.drop(columns=["TARGET", "COUNTRY"])
y_all = df["TARGET"]


X_train, X_test, y_train, y_test = train_test_split(X_all, y_all, test_size=0.2, random_state=42)

lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred_train = lr.predict(X_train)
y_pred_test  = lr.predict(X_test)

def spearman_corr(y_true, y_pred):
    return spearmanr(y_true, y_pred).correlation

print("Corrélation (Spearman) train : {:.1f}%".format(100 * spearman_corr(y_train, y_pred_train)))
print("Corrélation (Spearman) test  : {:.1f}%".format(100 * spearman_corr(y_test,  y_pred_test)))

ValueError: Input X contains NaN.
LinearRegression does not accept missing values encoded as NaN natively. For supervised learning, you might want to consider sklearn.ensemble.HistGradientBoostingClassifier and Regressor which accept missing values encoded as NaNs natively. Alternatively, it is possible to preprocess the data, for instance by using an imputer transformer in a pipeline or drop samples with missing values. See https://scikit-learn.org/stable/modules/impute.html You can find a list of all estimators that handle NaN values at the following page: https://scikit-learn.org/stable/modules/impute.html#estimators-that-handle-nan-values

#### Modèle sur df_fr (En utilant le df contenant les colonnes from PCA plus colonnes seuils)

Et regression pour le moment


#

#### Modèle sur df_de (En utilant le de contenant les colonnes from PCA plus colonnes seuils)

Regression aussi

#### Assemblage des deux modèles et train/test comme le modèle de base pour comparer 
Il faudra penser, si on a le temps à faire du k-fold pour éviter l'overfitting, genre on divise en 5 morceau et à chaques fois on change les morceaux qui entrainent et qui test et on voit si le modèle généralise bien.

#### Autres modèles à faire ensuite

### Polynomiale Regression (vue lab2)

### Decision Tree Regressor (vue lab2)